In [3]:
import face_recognition
from sklearn import svm
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Training the SVC classifier

# The training data would be all the face encodings from all the known images and the labels are their names
encodings = []
names = []

# Training directory
train_cloaked_dir = os.listdir('train_cloaked')
train_uncloaked_dir = os.listdir('train_uncloaked')
test_dir = os.listdir("val")

## Train with cloaked img

In [4]:
# Loop through each person in the training directory
for person in train_cloaked_dir:
    pix = os.listdir("train_cloaked/" + person)
    if person == "obama":
        pass

    # Loop through each training image for the current person
    for person_img in pix:

        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("train_cloaked/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

# Create and train the SVC classifier
clf_cloaked = svm.SVC(gamma='scale', probability=True)
clf_cloaked.fit(encodings,names)

ben_afflek/httpssmediacacheakpinimgcomxeeedcacddeccccacfjpg.jpg was skipped and can't be used for training
madonna/httpssmediacacheakpinimgcomxafafedebcbjpg.jpg was skipped and can't be used for training
elton_john/httpsdfrpwlycscloudfrontnetdeltonjohnjpg.jpg was skipped and can't be used for training
elton_john/httpsticketmnettmenusdamabbacdaebeafdbbCUSTOMjpg.jpg was skipped and can't be used for training
mindy_kaling/httpssmediacacheakpinimgcomxeedccadejpg.jpg was skipped and can't be used for training
jerry_seinfeld/httpwwweonlinecomresizewwweonlinecomeolimagesEntireSitersxjerryseinfeldjulialouisdreyfusjwjpg.jpg was skipped and can't be used for training


SVC(probability=True)

## Train with uncloaked img

In [5]:
# Loop through each person in the training directory
for person in train_uncloaked_dir:
    pix = os.listdir("train_uncloaked/" + person)
    if person == "obama_cloaked":
        pass

    # Loop through each training image for the current person
    for person_img in pix:

        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("train_uncloaked/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")

# Create and train the SVC classifier
clf_uncloaked = svm.SVC(gamma='scale', probability=True)
clf_uncloaked.fit(encodings,names)

ben_afflek/httpssmediacacheakpinimgcomxeeedcacddeccccacfjpg.jpg was skipped and can't be used for training
madonna/httpssmediacacheakpinimgcomxafafedebcbjpg.jpg was skipped and can't be used for training
elton_john/httpsdfrpwlycscloudfrontnetdeltonjohnjpg.jpg was skipped and can't be used for training
elton_john/httpsticketmnettmenusdamabbacdaebeafdbbCUSTOMjpg.jpg was skipped and can't be used for training
mindy_kaling/httpssmediacacheakpinimgcomxeedccadejpg.jpg was skipped and can't be used for training
jerry_seinfeld/httpwwweonlinecomresizewwweonlinecomeolimagesEntireSitersxjerryseinfeldjulialouisdreyfusjwjpg.jpg was skipped and can't be used for training


SVC(probability=True)

## Test on uncloaked

In [6]:
accuracy_uncloaked = 0
accuracy_cloaked = 0
total_pred = 0

columns = ['ben_afflek', 'elton_john','jerry_seinfeld','madonna','mindy_kaling','obama']
proba_cloaked_df = pd.DataFrame(columns= columns)
proba_uncloaked_df = pd.DataFrame(columns= columns)
# Loop through each person in the training directory

pix = os.listdir("val/obama")

# Loop through each training image for the current person
for person_img in pix:
    # Get the face encodings for the face in each image file
    test_image = face_recognition.load_image_file("val/obama/" + person_img)
    face_locations = face_recognition.face_locations(test_image)
    no = len(face_locations)

    for i in range(no):
        test_image_enc = face_recognition.face_encodings(test_image)[i]

        proba_uncloaked = clf_uncloaked.predict_proba([test_image_enc])
        proba_uncloaked_df = proba_uncloaked_df.append(pd.DataFrame(proba_uncloaked, columns= columns), ignore_index = True)

        proba_cloaked = clf_cloaked.predict_proba([test_image_enc])
        proba_cloaked_df = proba_cloaked_df.append(pd.DataFrame(proba_cloaked, columns= columns), ignore_index = True)

        predict_name_uncloaked = clf_uncloaked.predict([test_image_enc])
        predict_name_cloaked = clf_cloaked.predict([test_image_enc])

        if predict_name_uncloaked == "obama":
            accuracy_uncloaked +=1
        if predict_name_cloaked == "obama":
            accuracy_cloaked +=1

        total_pred +=1
    
print(f"Accuracy on uncloaked training = {accuracy_uncloaked * 100 / total_pred}%")

    
print(f"Accuracy on cloaked training = {accuracy_cloaked * 100 / total_pred}%")

Accuracy on uncloaked training = 100.0%
Accuracy on cloaked training = 100.0%


In [7]:
proba_uncloaked_df

,ben_afflek,elton_john,jerry_seinfeld,madonna,mindy_kaling,obama
0,0.013483,0.032334,0.02347,0.013031,0.012842,0.904839
1,0.013088,0.037948,0.024915,0.015224,0.019872,0.888953
2,0.019897,0.052145,0.028685,0.026361,0.033676,0.839235
3,0.011767,0.017102,0.016042,0.011377,0.012124,0.931588
4,0.032606,0.064315,0.06458,0.024213,0.020381,0.793906
5,0.010538,0.026957,0.017666,0.012301,0.014209,0.918329
6,0.017347,0.072931,0.025716,0.017148,0.022658,0.8442
7,0.034745,0.069389,0.06154,0.026316,0.021995,0.786015
8,0.020735,0.036212,0.026985,0.0167,0.036322,0.863045


In [8]:
proba_cloaked_df

,ben_afflek,elton_john,jerry_seinfeld,madonna,mindy_kaling,obama
0,0.020778,0.039938,0.032956,0.018556,0.021505,0.866267
1,0.020907,0.047644,0.033534,0.020913,0.029819,0.847183
2,0.027957,0.062331,0.037394,0.032413,0.047442,0.792464
3,0.018962,0.025956,0.025842,0.016558,0.02052,0.892161
4,0.046598,0.079647,0.080632,0.031409,0.032512,0.729201
5,0.01735,0.03895,0.027347,0.018893,0.025262,0.872198
6,0.025832,0.094354,0.037749,0.022512,0.033295,0.786258
7,0.048551,0.086633,0.078078,0.033191,0.034411,0.719137
8,0.031884,0.048383,0.040483,0.025484,0.051742,0.802024
